please mount google drive before you begin

# Part I: Set-up and Data Processing

In [25]:
cd /content/drive/MyDrive/DEX_Uniswap_Layer2/Poly

/content/drive/MyDrive/DEX_Uniswap_Layer2/Poly


In [26]:
pwd

'/content/drive/MyDrive/DEX_Uniswap_Layer2/Poly'

In [28]:
# importing pandas libraries and
# glob module
import pandas as pd

In [29]:
#@title path
data = "/content/drive/MyDrive/DEX_Uniswap_Layer2/Poly/Amount" #@param {type:"string"}

In [30]:
import os

# folder path
dir_path = r'/content/drive/MyDrive/DEX_Uniswap_Layer2/Poly/Amount'

# list file and directories
res = os.listdir(dir_path)
print(res)

['Poly_Amount_Feb.csv', 'Poly_Amount_Mar.csv', 'Poly_Amount_Apr.csv', 'Poly_Amount_June.csv', 'Poly_Amount_July.csv', 'Poly_Amount_May.csv', 'Poly_Amount_Aug.csv', 'Poly_Amount_Oct.csv', 'Poly_Amount_Dec2021.csv', 'Poly_Amount_Dec.csv', 'Poly_Amount_Sep.csv', 'Poly_Amount_Nov.csv', 'Poly_Amount_Jan.csv']


In [31]:
df1 = pd.read_csv(dir_path+"/"+"Poly_Amount_Jan.csv",index_col="Unnamed: 0")
df1.head()

,Timestamp,Amount_USD
38944,2022-01-01 00:00:01,1277.224818
276,2022-01-01 00:00:01,1277.224818
57026,2022-01-01 00:00:01,2498.041299
18358,2022-01-01 00:00:01,2498.041299
62578,2022-01-01 00:00:17,51.151019


In [32]:
files = res

In [33]:
df =df1
from tqdm import tqdm
for file in tqdm(files):
    # combining multiple excel worksheets
    # into single data frames
    df = pd.concat([df, pd.read_csv(dir_path+"/"+file,index_col="Unnamed: 0")],
                   ignore_index=True, sort=False)

100%|██████████| 13/13 [00:36<00:00,  2.79s/it]


In [ ]:
df_Polygon = df
df_Polygon.tail()

,Timestamp,Amount_USD
31526501,2022-01-31 23:59:50,41.312080
31526502,2022-01-31 23:59:52,73.082361
31526503,2022-01-31 23:59:52,73.121584
31526504,2022-01-31 23:59:52,45.035286
31526505,2022-01-31 23:59:52,45.036830


In [ ]:
df_Polygon['Timestamp']=pd.to_datetime(df_Polygon['Timestamp'])
df_Polygon.dtypes

Timestamp     datetime64[ns]
Amount_USD           float64
dtype: object

In [ ]:
df_Polygon=df_Polygon.sort_values(by='Timestamp')
df_Polygon

,Timestamp,Amount_USD
20432580,2021-12-20 14:27:40,0.000000
20432579,2021-12-20 14:27:40,0.000000
20432581,2021-12-21 08:20:43,84.811697
20432582,2021-12-21 08:20:43,0.000000
20432583,2021-12-21 08:20:55,10.805575
...,...,...
22973173,2022-12-31 23:59:36,0.011947
22973174,2022-12-31 23:59:36,0.075237
22973175,2022-12-31 23:59:52,1.790000
22973177,2022-12-31 23:59:54,0.105331


In [ ]:
df_Polygon.to_csv("Polygon.csv")

# Part II Calculating Daily Transaction Volume

In [ ]:
df_Polygon = pd.read_csv("Polygon.csv",index_col="Unnamed: 0")
df_Polygon['Timestamp']=pd.to_datetime(df_Polygon['Timestamp'])
df_Polygon.dtypes

Timestamp     datetime64[ns]
Amount_USD           float64
dtype: object

In [ ]:
df_Polygon.tail()

,Timestamp,Amount_USD
22973173,2022-12-31 23:59:36,0.011947
22973174,2022-12-31 23:59:36,0.075237
22973175,2022-12-31 23:59:52,1.790000
22973177,2022-12-31 23:59:54,0.105331
22973176,2022-12-31 23:59:54,0.752557


In [ ]:
df_Polygon['date'] =df_Polygon['Timestamp'].dt.date
df_Polygon

,Timestamp,Amount_USD,date
20432580,2021-12-20 14:27:40,0.000000,2021-12-20
20432579,2021-12-20 14:27:40,0.000000,2021-12-20
20432581,2021-12-21 08:20:43,84.811697,2021-12-21
20432582,2021-12-21 08:20:43,0.000000,2021-12-21
20432583,2021-12-21 08:20:55,10.805575,2021-12-21
...,...,...,...
22973173,2022-12-31 23:59:36,0.011947,2022-12-31
22973174,2022-12-31 23:59:36,0.075237,2022-12-31
22973175,2022-12-31 23:59:52,1.790000,2022-12-31
22973177,2022-12-31 23:59:54,0.105331,2022-12-31


In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
import numpy as np
df_Polygon_volume=df_Polygon.groupby('date').Amount_USD.agg([np.min, np.max, np.sum,np.std, np.var,np.mean, np.median, percentile(75), percentile(25)]).reset_index()
df_Polygon_volume

,date,amin,amax,sum,std,var,mean,median,percentile_75,percentile_25
0,2021-12-20,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,2021-12-21,0.0,4991.537458,4.976985e+04,254.841078,6.494398e+04,48.414253,5.184673,18.946745,2.005832
2,2021-12-22,0.0,75070.855375,3.147078e+06,1108.946635,1.229763e+06,275.455448,68.103127,185.173283,19.660777
3,2021-12-23,0.0,49962.911639,2.462970e+07,1613.583053,2.603650e+06,547.253792,115.735044,380.901874,33.948723
4,2021-12-24,0.0,103699.250625,2.932176e+07,2127.453243,4.526057e+06,661.338483,112.896571,420.093703,38.958784
...,...,...,...,...,...,...,...,...,...,...
372,2022-12-27,0.0,285192.584744,3.428509e+07,2423.448608,5.873103e+06,571.837470,28.892993,301.247061,1.879436
373,2022-12-28,0.0,80022.450391,2.953848e+07,1811.203249,3.280457e+06,537.669326,25.373004,313.938164,1.010000
374,2022-12-29,0.0,85819.410959,2.986198e+07,1731.795053,2.999114e+06,487.057385,17.917506,223.529508,0.790304
375,2022-12-30,0.0,247800.033354,1.793022e+07,1751.686074,3.068404e+06,376.859506,12.931272,139.894935,0.768387


In [ ]:
df_Polygon_volume.to_csv("Polygon_volume.csv")

In [ ]:
import numpy as np
df_Polygon.describe()

,Amount_USD
count,3.152651e+07
mean,1.079894e+03
std,3.655091e+03
min,0.000000e+00
25%,2.037665e+01
50%,1.403438e+02
75%,9.228344e+02
max,2.087537e+06


# Part III Calculating Decentralized Index

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import pytz

In [ ]:
def index(df):
    df['Amount_USD'] = df['Amount_USD'].astype(float)
    df['prop'] = df['Amount_USD']/df['Amount_USD'].sum()
    H = -df['prop']*np.log2(df['prop'])
    E = H.sum()
    V = 2**E
    return V

def HHI(df):
    df['value'] = df['value'].astype(float)
    df['prop'] = (df['value']/df['value'].sum())*100
    SQ = df['prop']**2
    V = SQ.sum()
    return V

def gini_coefficient(values):
    """
    Compute Gini coefficient of an array of values
    """
    # Check if list is empty
    if len(values) == 0:
        return None

    # Sort the values
    values = sorted(values)

    # Compute the cumulative sum of the values and their indices
    cum_values = [0]
    for i in values:
        cum_values.append(cum_values[-1] + i)

    n = len(values)

    # Calculate Gini coefficient using the formula
    numer = sum([(2 * (i+1) - n - 1 ) * values[i] for i in range(n)])
    denom = n * sum(values)

    if denom == 0:
        return None  # Avoid division by zero

    gini = numer / denom

    return gini

def nakamoto(df):
    df.sort_values(by=['Amount_USD'],ascending=False,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df.set_index(keys=df.index.values+1,inplace=True)
    df['prop'] = df['Amount_USD']/df['Amount_USD'].sum()
    df['cumprop'] = df['prop'].cumsum(axis=0)
    try:
        V = df[df['cumprop'] > 0.5].index[0]
        return V
    except:
        return 0

"""
def index_calc(data):

            :param data: dataframe containing transaction Amount_USD and timestamp of the transaction (Timestamp)
            :type data: pandas.DataFrame object
            :param start: the start date of calculation 00:00:00 UTC
            :type start: datetime.date object
            :param end: the end date of calculation 23:59:59 UTC
            :type end: datetime.date object
            :param index_type: the metrics used for measuring decentralization, default is our index
            :type index_type: index function

    returns pandas dataframe with decentralization index Amount_USD and date
"""
from tqdm import tqdm
def index_calc(data,  start, end, index_type = index):
    duration= pd.date_range(start=start, end=end)
    days = np.size(duration)
    IndexValues = pd.DataFrame(np.zeros(days), columns=['val'])
    IndexValues['date'] = duration

    for i in tqdm(range(0, days)):
        start_date = start + timedelta(days=i)
        end_date = start_date + timedelta(days=1)
        IndexValues.loc[i,'val'] = index_type(data[(data['date'] >= start_date) & (data['date'] < end_date)].copy())

    return IndexValues

In [ ]:
#input transaction data and desired date range
#returns index value 1 if specification range exceeds data
start = date(2021,12,20)
end = date(2022,12,31)
results = index_calc(df_Polygon,start,end)

In [ ]:
results.to_csv("Polygon_Entropy.csv")

In [ ]:
results

,val,date
0,1.000000,2021-12-20
1,103.490202,2021-12-21
2,2619.568232,2021-12-22
3,11190.695825,2021-12-23
4,10303.275727,2021-12-24
...,...,...
372,10071.972867,2022-12-27
373,10006.920215,2022-12-28
374,9995.969468,2022-12-29
375,6765.091943,2022-12-30


In [ ]:
#specify alternate index metric: HHI
#returns index value 0 if specification range exceeds data
start = date(2021,12,20)
end = date(2022,12,31)
results_HHI = index_calc(df_Polygon,start,end,HHI)
results_HHI

100%|██████████| 377/377 [18:40<00:00,  2.97s/it]


,val,date
0,0.000000,2021-12-20
1,0.027899,2021-12-21
2,0.001506,2021-12-22
3,0.000215,2021-12-23
4,0.000256,2021-12-24
...,...,...
372,0.000316,2022-12-27
373,0.000225,2022-12-28
374,0.000223,2022-12-29
375,0.000475,2022-12-30


In [ ]:
results_HHI.to_csv("Polygon_HHI.csv")

In [ ]:
#specify alternate index metric: gini
#returns index value 0 if specification range exceeds data
start = date(2021,12,20)
end = date(2022,12,31)
results_gini = index_calc(df_Polygon,start,end,gini)
results_gini

100%|██████████| 377/377 [18:49<00:00,  3.00s/it]


,val,date
0,0.000000,2021-12-20
1,0.876249,2021-12-21
2,0.777360,2021-12-22
3,0.783262,2021-12-23
4,0.798430,2021-12-24
...,...,...
372,0.858410,2022-12-27
373,0.851686,2022-12-28
374,0.867507,2022-12-29
375,0.880108,2022-12-30


In [ ]:
results_gini.to_csv("Polygon_gini.csv")

In [ ]:
#specify alternate index metric: nakamoto coefficient
#returns index value 0 if specification range exceeds data
start = date(2021,12,20)
end = date(2022,12,31)
results_nakamoto = index_calc(df_Polygon,start,end,nakamoto)
results_nakamoto

100%|██████████| 377/377 [18:58<00:00,  3.02s/it]


,val,date
0,0.0,2021-12-20
1,15.0,2021-12-21
2,499.0,2021-12-22
3,2156.0,2021-12-23
4,2067.0,2021-12-24
...,...,...
372,2182.0,2022-12-27
373,2208.0,2022-12-28
374,2162.0,2022-12-29
375,1414.0,2022-12-30


In [ ]:
results_nakamoto.to_csv("Polygon_nakamoto.csv")